Basic pedestrians crossing dataset generation + pose projection overlay
=======================================================================

Random generators are seeded, so unless code/settings are changed, the same results will be obtained.

In [ ]:
OUTPUT_DIR = '/outputs/scenarios/datasets/crossing'

In [ ]:
import os
import shutil

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

In [ ]:
from pedestrians_scenarios.skeleton import main

main([
    f"--outputs-dir={OUTPUT_DIR}",
    "--number-of-clips=8",
    "--clip-length-in-frames=600",
    "--batch-size=4",
])

In [ ]:
import pandas as pd
import ast

from pedestrians_scenarios.karma.utils.conversions import convert_flat_list_to_vector3d, convert_flat_list_to_transform
from pedestrians_scenarios.karma.pose.pose_dict import convert_flat_list_to_pose_dict, convert_flat_list_to_pose_2d_dict

dataset = pd.read_csv(
    os.path.join(OUTPUT_DIR, 'data.csv'),
    index_col=['id', 'camera.idx', 'frame.idx', 'pedestrian.idx'],
    converters={
        'camera.transform': lambda x: convert_flat_list_to_transform(ast.literal_eval(x)),
        'pedestrian.spawn_point': lambda x: convert_flat_list_to_transform(ast.literal_eval(x)),
        'frame.pedestrian.transform': lambda x: convert_flat_list_to_transform(ast.literal_eval(x)),
        'frame.pedestrian.velocity': lambda x: convert_flat_list_to_vector3d(ast.literal_eval(x)),
        'frame.pedestrian.pose.world': lambda x: convert_flat_list_to_pose_dict(ast.literal_eval(x)),
        'frame.pedestrian.pose.component': lambda x: convert_flat_list_to_pose_dict(ast.literal_eval(x)),
        'frame.pedestrian.pose.relative': lambda x: convert_flat_list_to_pose_dict(ast.literal_eval(x)),
        'frame.camera.pose': lambda x: convert_flat_list_to_pose_2d_dict(ast.literal_eval(x))
    }
)

In [ ]:
# extract the correct frame images from the recordings
# for each frame, overlay the pose on the image
# an save as new video

import numpy as np

from pedestrians_scenarios.karma.pose.skeleton import CARLA_SKELETON
from pedestrians_scenarios.karma.renderers.source_videos_renderer import SourceVideosRenderer

os.makedirs(os.path.join(OUTPUT_DIR, 'poses'), exist_ok=True)
renderer = SourceVideosRenderer(data_dir=os.path.join(OUTPUT_DIR, 'clips'))


for (index_label, row_group) in dataset.groupby(level=['id', 'camera.idx', 'pedestrian.idx']):
    clip_id = index_label[0]
    video_id = row_group.iloc[0]['camera.recording'].replace('clips/','').replace('.mp4', '') # this is derived from camera.idx and other values
    pedestrian_id = index_label[2]
    projections = []
    for _, row in row_group.iterrows():
        projection = np.array([[v.x, v.y] for v in row['frame.camera.pose'].values()], dtype=np.float32)
        projections.append(projection)
    frames = renderer.render_clip(
        video_id=video_id,
        pedestrian_id=pedestrian_id,
        clip_id=clip_id,
        start_frame=0,
        end_frame=len(projections),
        bboxes=None,
        skeletons=[{
            'keypoints': projections,
            'color': None,
            'type': CARLA_SKELETON
        }]
    )
    renderer.frames_to_video(frames, video_id, os.path.join(OUTPUT_DIR, 'poses'))

    

In [ ]:
import glob
from IPython.display import HTML

videos_list = glob.glob('/outputs/scenarios/datasets/crossing/poses/*.mp4', recursive=True)

In [ ]:
shown_videos_list = videos_list[:4] # limit number of videos so that jupyter notebook doesn't crash

html_source = []
for video_path in shown_videos_list:
    video_tag = '<video width="100%" controls src="{}" type="video/mp4"></video>'.format(video_path)
    html_source.append(video_tag)

HTML(data='<br />\n'.join(html_source))